In [6]:
class ParameterValues:
    def __init__(self, epsnx, alfax, betax, epsny, alfay, betay, epsnz, alfaz, betaz):
        self.epsnx = epsnx
        self.alfax = alfax
        self.betax = betax
        self.epsny = epsny
        self.alfay = alfay
        self.betay = betay
        self.epsnz = epsnz
        self.alfaz = alfaz
        self.betaz = betaz

In [7]:
class Particle:
    def __init__(self, Nseed, iq, dt, dW, x, x_prime, y, y_prime):
        self.Nseed = Nseed
        self.iq = iq
        self.dt = dt
        self.dW = dW
        self.x = x
        self.x_prime = x_prime
        self.y = y
        self.y_prime = y_prime
        self.z = 0

    def __repr__(self):
        return f"Particle(Nseed={self.Nseed}, iq={self.iq}, dt={self.dt}, dW={self.dW}, x={self.x}, x'={self.x_prime}, y={self.y}, y'={self.y_prime})"

    def propagate(self, distance):
        import math
        # print(str(self.x) + " " + str(self.y), end=" ")
        # convert angles from milliradians to radians
        angle_x = self.x_prime / 1000 
        angle_y = self.y_prime / 1000
        # print("->", end=" ")
        # calculate new positions
        self.x += distance * math.tan(angle_x)
        self.y += distance * math.tan(angle_y)
        # print(str(self.x) + " " + str(self.y))

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

class Hole:
    def __init__(self, center_x, center_y, center_z, diameter):
        self.center_x = center_x
        self.center_y = center_y
        self.center_z = center_z
        self.diameter = diameter

    def __repr__(self):
        return f"Hole(center_x={self.center_x}, center_y={self.center_y}, center_z={self.center_z}, diameter={self.diameter})"
    
class Grid:
    def __init__(self, size_x, size_y, size_z, hole_diameter, separation):
        self.size_x = size_x
        self.size_y = size_y
        self.size_z = size_z
        self.hole_diameter = hole_diameter
        self.separation = separation
        self.holes = self.create_3d_grid()

    def create_3d_grid(self):
        holes = []
        center_x_offset = ((self.size_x - 1) * self.separation) / 2
        center_y_offset = ((self.size_y - 1) * self.separation) / 2
        center_z_offset = ((self.size_z - 1) * self.separation) / 2
        for x in range(self.size_x):
            for y in range(self.size_y):
                for z in range(self.size_z):
                    hole = Hole(center_x=x*self.separation - center_x_offset, 
                                 center_y=y*self.separation - center_y_offset, 
                                 center_z=z*self.separation - center_z_offset, 
                                 diameter=self.hole_diameter)
                    holes.append(hole)
        return holes

    def plot3D(self, ax=None):
        xs = [hole.center_x for hole in self.holes]
        ys = [hole.center_y for hole in self.holes]
        zs = [hole.center_z for hole in self.holes]
        
        if ax is None:
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
        
        ax.scatter(xs, ys, zs, c='red')

    def add_to_plot3D(self, ax):
        self.plot3D(ax)

    def add_to_plot_2d(self, ax):
        # Add the grid to the 2D plot
        import matplotlib.patches as patches
        for hole in self.holes:
            ax.plot(hole.center_x, hole.center_y, 'ro')

            # Add circle representing the hole's circumference
            circle = patches.Circle((hole.center_x, hole.center_y), radius=hole.diameter/2, edgecolor='red', facecolor='none')
            ax.add_patch(circle)


    def is_point_in_a_hole(self, point):
        x1, y1, z1 = point
        hole_radius = self.hole_diameter / 2

        # Calculate the indices of the hole the point would belong to if it was in a hole
        x_index = round(x1 / self.separation)
        y_index = round(y1 / self.separation)
        z_index = round(z1 / self.separation)

        # Calculate the center of that hole
        x_center = x_index * self.separation
        y_center = y_index * self.separation
        z_center = z_index * self.separation

        # Check if the point is inside the hole
        # return ((x_center - x1)**2 + (y_center - y1)**2 + (z_center - z1)**2) <= hole_radius**2
        return (x_center - hole_radius <= x1 <= x_center + hole_radius and
                y_center - hole_radius <= y1 <= y_center + hole_radius and
                z_center - hole_radius <= z1 <= z_center + hole_radius)


In [8]:
import os
def write_folder_names_to_file(data_folder, cache_file):
    # Open the cache file in append mode
    with open(cache_file, "w") as cache:
        # Walk through the directory
        for _, dirs, _ in os.walk(data_folder):
            # For each directory in the directory
            for dir in dirs:
                # Write the directory name to the cache file
                cache.write(dir + "\n")

# data_folder = "E:\\data"
# cache_file = "tested_params.txt"
# write_folder_names_to_file(data_folder, cache_file)


In [9]:
import subprocess
import time
import os
import shutil
import fileinput
import re
import numpy as np

def kill_process(process_name):
    try:
        subprocess.run(['taskkill', '/F', '/IM', process_name], check=True)
        print(f"{process_name} has been terminated.")
    except subprocess.CalledProcessError as e:
        print(f"Error occurred while trying to kill {process_name}: {str(e)}")
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")

class ParameterValues:
    def __init__(self, epsnx, alfax, betax, epsny, alfay, betay, epsnz, alfaz, betaz):
        self.epsnx = epsnx
        self.alfax = alfax
        self.betax = betax
        self.epsny = epsny
        self.alfay = alfay
        self.betay = betay
        self.epsnz = epsnz
        self.alfaz = alfaz
        self.betaz = betaz

parameter_ranges = {
    "alfax": np.random.uniform(-5.0, 5.0, 1000).tolist(), 
    "betax": np.random.uniform(50.0, 500.0, 1000).tolist(),
}

num_iterations = len(parameter_ranges["alfax"])
tested_params_file = "tested_params.txt"  # File to store tested parameters

# Loop over all sets of parameters
for i in range(num_iterations):
    try:
        # Every 8th iteration or the first one
        if (i + 1) % 8 == 0 or True:
            # Moving files from current directory to another directory
            current_data_folder = os.path.join(os.getcwd(), "data")
            new_data_folder = "E:\\data"
            
            for folder_name in os.listdir(current_data_folder):
                folder_path = os.path.join(current_data_folder, folder_name)
                if os.path.isdir(folder_path):
                    destination_folder = os.path.join(new_data_folder, folder_name)
                    if os.path.exists(destination_folder):
                        shutil.rmtree(destination_folder)
                    shutil.copytree(folder_path, destination_folder)
                    shutil.rmtree(folder_path)
                    
        alfax = parameter_ranges["alfax"][i]
        betax = parameter_ranges["betax"][i]

        VALUES = ParameterValues(
            epsnx=0.1,
            alfax=alfax,
            betax=betax,
            epsny=0.1,
            alfay=-0.55,
            betay=170.0,
            epsnz=5.00,
            alfaz=0.10,
            betaz=10.0
        )

        # Forming filename from parameters
        filename = (f"epsnx{VALUES.epsnx:.2f}_alfax{VALUES.alfax:.2f}_betax{VALUES.betax:.2f}_" +
                    f"epsny{VALUES.epsny:.2f}_alfay{VALUES.alfay:.2f}_betay{VALUES.betay:.2f}_" +
                    f"epsnz{VALUES.epsnz:.2f}_alfaz{VALUES.alfaz:.2f}_betaz{VALUES.betaz:.2f}")

        # Check if filename has already been tested
        with open(tested_params_file, "r") as file:
            lines = file.readlines()
            if filename + "\n" in lines:
                print(f"Skipping iteration {i}. Parameters already tested.")
                continue
        
        track_file = "sim/track.dat"

        with open(track_file, "r") as file:
            lines = file.readlines()

        parameter_lines = lines[14:17]
        for j, line in enumerate(parameter_lines):
            if any(param in line for param in parameter_ranges.keys()):
                parts = line.split(',')
                for k, part in enumerate(parts):
                    for param_name, param_value in parameter_ranges.items():
                        if param_name in part:
                            pattern = r"=(.*?)d0"
                            matches = re.findall(pattern, part)
                            new_value = str(param_value[i])
                            parts[k] = part.replace(matches[0], new_value)
                            break
                parameter_lines[j] = ",".join(parts)

        lines[14:17] = parameter_lines

        with open(track_file, "w") as file:
            file.writelines(lines)

        exe_path = "sim/TRACKv39.exe"
        exe_dir = os.path.dirname(exe_path)
        log_file = "sim/log.out"
        output_dir = os.path.join(
            os.getcwd(),
            "data",
            filename
        )

        process = subprocess.Popen(exe_path, cwd=exe_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        log_file_exists = os.path.isfile(log_file)
        
        start_time = time.time()

        while not log_file_exists:
            time.sleep(1)
            log_file_exists = os.path.isfile(log_file)

        running = True
        while running:
            if log_file_exists:
                with open(log_file, "r") as file:
                    lines = file.readlines()
                    if len(lines) >= 8 and lines[7].strip():
                        running = False

            if time.time() - start_time > 500:  # Timeout
                print("Process exceeded time limit. Restarting...")
                process.kill()
                running = False
                continue
                        
        process.terminate()
        kill_process("TRACKv39.exe")

        os.makedirs(output_dir, exist_ok=True)

        shutil.copy("sim/track.dat", os.path.join(output_dir, "track.dat"))
        shutil.move(log_file, os.path.join(output_dir, "log.out"))
        shutil.move("sim/beam.out", os.path.join(output_dir, "beam.out"))
        shutil.move("sim/coord.out", os.path.join(output_dir, "coord.out"))
        print(i)

        # Write filename to file of tested parameters
        with open(tested_params_file, "a") as file:
            file.write(filename + "\n")
        
        # Every couple of iterations wipe and refill
        if (i % 10 == 0):
            data_folder = "E:\\data"
            cache_file = "tested_params.txt"
            write_folder_names_to_file(data_folder, cache_file)

        # print("Started generating image.")

        # import os
        # import re
        # import matplotlib.pyplot as plt
        # import numpy as np

        # subfolder = "data"
        # os.makedirs("images", exist_ok=True)

        # folder_path = output_dir
        # if os.path.isdir(folder_path):
        #     coord_file = os.path.join(folder_path, "coord.out")
        #     if os.path.isfile(coord_file):
        #         data = []
        #         with open(coord_file, 'r') as file:
        #             next(file)
        #             for line in file:
        #                 line = line.strip()
        #                 if line:
        #                     values = line.split()
        #                     particle = Particle(
        #                         Nseed=int(values[0]),
        #                         iq=int(values[1]),
        #                         dt=float(values[2]),
        #                         dW=float(values[3]),
        #                         x=float(values[4]),
        #                         x_prime=float(values[5]),
        #                         y=float(values[6]),
        #                         y_prime=float(values[7])
        #                     )
        #                     data.append(particle)

        #         size_x, size_y, size_z = 21, 21, 1
        #         hole_diameter = 0.01
        #         separation = 0.3

        #         grid = Grid(size_x, size_y, size_z, hole_diameter, separation)

        #         particles_in_holes = []
        #         for particle in data:
        #             point = (particle.x, particle.y, particle.z)
        #             if grid.is_point_in_a_hole(point):
        #                 particles_in_holes.append(particle)

        #         for particle in particles_in_holes:
        #             particle.propagate(10)

        #         if particles_in_holes:
        #             x_in_holes = [particle.x for particle in particles_in_holes]
        #             y_in_holes = [particle.y for particle in particles_in_holes]

        #         bins_hist = 60

        #         edges_hist = np.linspace(-3, 3, bins_hist + 1)

        #         H, _, _ = np.histogram2d(x_in_holes, y_in_holes, bins=[edges_hist, edges_hist], density=True)

        #         fig, ax = plt.subplots(figsize=(6, 6))

        #         pcm = ax.pcolormesh(edges_hist, edges_hist, H.T, cmap='inferno')

        #         ax.set_xticks([])
        #         ax.set_yticks([])
        #         ax.set_frame_on(False)

        #         plt.axis('square')

        #         filename = (f"epsnx{VALUES.epsnx:.2f}_alfax{VALUES.alfax:.2f}_betax{VALUES.betax:.2f}_" +
        #         f"epsny{VALUES.epsny:.2f}_alfay{VALUES.alfay:.2f}_betay{VALUES.betay:.2f}_" +
        #         f"epsnz{VALUES.epsnz:.2f}_alfaz{VALUES.alfaz:.2f}_betaz{VALUES.betaz:.2f}")

        #         plt.savefig(f"images/{filename}.png", dpi=64, bbox_inches='tight', pad_inches=0)

        #         plt.close(fig)

        # print("Image generated successfully for the current dataset.")
        
        print("Program has terminated. Files copied to output directory:", output_dir)
    except:
        continue


Error occurred while trying to kill TRACKv39.exe: Command '['taskkill', '/F', '/IM', 'TRACKv39.exe']' returned non-zero exit status 1.
0
Program has terminated. Files copied to output directory: c:\Users\ianja\REPOS\Pepperpot\data\epsnx0.10_alfax-4.60_betax101.12_epsny0.10_alfay-0.55_betay170.00_epsnz5.00_alfaz0.10_betaz10.00
Error occurred while trying to kill TRACKv39.exe: Command '['taskkill', '/F', '/IM', 'TRACKv39.exe']' returned non-zero exit status 128.
1
Program has terminated. Files copied to output directory: c:\Users\ianja\REPOS\Pepperpot\data\epsnx0.10_alfax1.36_betax87.75_epsny0.10_alfay-0.55_betay170.00_epsnz5.00_alfaz0.10_betaz10.00
Error occurred while trying to kill TRACKv39.exe: Command '['taskkill', '/F', '/IM', 'TRACKv39.exe']' returned non-zero exit status 128.
2
Program has terminated. Files copied to output directory: c:\Users\ianja\REPOS\Pepperpot\data\epsnx0.10_alfax-4.69_betax374.97_epsny0.10_alfay-0.55_betay170.00_epsnz5.00_alfaz0.10_betaz10.00
Error occurred

In [ ]:
import os
import re

# Specify the directory
directory = 'images'

# Iterate through every file in the directory
for filename in os.listdir(directory):
    # Use regular expression to find underscores between numbers and replace them with a dot
    new_filename = re.sub(r'(\d)_(\d)', r'\1.\2', filename)
    
    try:
        # Rename the file
        os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))
    except:
        pass
print("File renaming completed!")


File renaming completed!


In [ ]:
print("Started generating images.")

import os
import re
import matplotlib.pyplot as plt
import numpy as np

subfolder = "E:/data"
os.makedirs("images", exist_ok=True)
i = 0

for foldername in os.listdir(subfolder):
    print(i)
    i += 1
    folder_path = os.path.join(subfolder, foldername)
    image_path = os.path.join("images", f"{foldername}.png")

    # Check if image already exists
    if os.path.exists(image_path):
        print(f"Image {foldername}.png already exists, skipping...")
        continue

    if os.path.isdir(folder_path):
        coord_file = os.path.join(folder_path, "coord.out")
        if os.path.isfile(coord_file):
            data = []
            print(foldername)
            with open(coord_file, 'r') as file:
                next(file)
                for line in file:
                    line = line.strip()
                    if line:
                        values = line.split()
                        particle = Particle(
                            Nseed=int(values[0]),
                            iq=int(values[1]),
                            dt=float(values[2]),
                            dW=float(values[3]),
                            x=float(values[4]),
                            x_prime=float(values[5]),
                            y=float(values[6]),
                            y_prime=float(values[7])
                        )
                        data.append(particle)

            size_x, size_y, size_z = 21, 21, 1
            hole_diameter = 0.01
            separation = 0.3

            grid = Grid(size_x, size_y, size_z, hole_diameter, separation)

            particles_in_holes = []
            for particle in data:
                point = (particle.x, particle.y, particle.z)
                if grid.is_point_in_a_hole(point):
                    particles_in_holes.append(particle)

            for particle in particles_in_holes:
                particle.propagate(10)

            if particles_in_holes:
                x_in_holes = [particle.x for particle in particles_in_holes]
                y_in_holes = [particle.y for particle in particles_in_holes]

            bins_hist = 60

            edges_hist = np.linspace(-3, 3, bins_hist + 1)

            H, _, _ = np.histogram2d(x_in_holes, y_in_holes, bins=[edges_hist, edges_hist], density=True)

            fig, ax = plt.subplots(figsize=(6, 6))

            pcm = ax.pcolormesh(edges_hist, edges_hist, H.T, cmap='inferno')

            ax.set_xticks([])
            ax.set_yticks([])
            ax.set_frame_on(False)

            plt.axis('square')

            plt.savefig(f"images/{foldername}.png", dpi=64, bbox_inches='tight', pad_inches=0)

            plt.close(fig)

    print("Images generated successfully.")

print("Program has terminated. Files copied to output directory:", output_dir)


Started generating images.
0
Image epsnx0.10_alfax2.15_betax54.53_epsny0.10_alfay-0.55_betay170.00_epsnz5.00_alfaz0.10_betaz10.00.png already exists, skipping...
1
Image epsnx0.10_alfax0.00_betax175.00_epsny0.10_alfay-0.55_betay170.00_epsnz5.00_alfaz0.10_betaz10.00.png already exists, skipping...
2
Image epsnx0.10_alfax0.45_betax368.95_epsny0.10_alfay-0.55_betay170.00_epsnz5.00_alfaz0.10_betaz10.00.png already exists, skipping...
3
Image epsnx0.10_alfax-0.76_betax69.79_epsny0.10_alfay-0.55_betay170.00_epsnz5.00_alfaz0.10_betaz10.00.png already exists, skipping...
4
Image epsnx0.10_alfax-1.00_betax150.00_epsny0.10_alfay-0.55_betay170.00_epsnz5.00_alfaz0.10_betaz10.00.png already exists, skipping...
5
Image epsnx0.10_alfax1.00_betax200.00_epsny0.10_alfay-0.55_betay170.00_epsnz5.00_alfaz0.10_betaz10.00.png already exists, skipping...
6
Image epsnx0.10_alfax1.03_betax264.12_epsny0.10_alfay-0.55_betay170.00_epsnz5.00_alfaz0.10_betaz10.00.png already exists, skipping...
7
Image epsnx0.10_alf

NameError: name 'output_dir' is not defined

In [ ]:
import os
import shutil

# Specify the path to the folder
path = "E:/data"

# Iterate over all subdirectories in the specified directory
for subdir, dirs, files in os.walk(path):
    # Iterate over all files in the current subdirectory
    for file in files:
        # If the file is 'coord.out'
        if file == 'coord.out':
            # Get the full path to the file
            file_path = os.path.join(subdir, file)
            # Get the size of the file in kilobytes
            file_size_kb = os.path.getsize(file_path) / 1024
            # If the size of the file is less than 100 KB
            if file_size_kb < 100:
                # Print the file path and size
                print(f"File '{file_path}' is {file_size_kb} KB.")
                # Delete the parent folder and all of its contents
                # shutil.rmtree(subdir)
                # print(f"Parent folder '{subdir}' has been deleted.")
                break
